# Clustering Crypto

In [135]:
# Initial imports
import pandas as pd
import hvplot.pandas
import numpy as np
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [136]:
# Load the crypto_data.csv dataset.
file_path = "./Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head()


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [137]:
crypto_df.count()

Unnamed: 0         1252
CoinName           1252
Algorithm          1252
IsTrading          1252
ProofType          1252
TotalCoinsMined     744
TotalCoinSupply    1252
dtype: int64

In [138]:
# check the data types
crypto_df.dtypes

Unnamed: 0          object
CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [139]:
# Checking for duplicates
print(f"Duplicate entries: {crypto_df.duplicated().sum()}")

Duplicate entries: 0


In [140]:
# keep a list of the names on the "Unnamed: 0" column. 
unnamed = crypto_df["Unnamed: 0"]
unnamed

0         42
1        365
2        404
3        611
4        808
        ... 
1247     XBC
1248    DVTC
1249    GIOT
1250    OPSC
1251    PUNK
Name: Unnamed: 0, Length: 1252, dtype: object

In [141]:
# Renamed "Unnamed: 0" column and then set it as the index
crypto_df = crypto_df.rename(columns={'Unnamed: 0': ''})
crypto_df = crypto_df.set_index('')
# crypto_df.sample()
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [142]:
# Keep all the cryptocurrencies that are being traded.
traded_crypto = crypto_df['IsTrading'] == True
crypto_df = crypto_df.loc[traded_crypto]
crypto_df.head(2)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,41.999954,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000


In [143]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df["Algorithm"].value_counts()

Scrypt                   394
X11                      182
SHA-256                  121
X13                       54
PoS                       42
                        ... 
VeChainThor Authority      1
Ouroboros                  1
POS 2.0                    1
Proof-of-BibleHash         1
TRC10                      1
Name: Algorithm, Length: 89, dtype: int64

In [144]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [145]:
# find null values 
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [146]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
crypto_df.head(2)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000


In [147]:
# check null values 
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [148]:
# Checking the row counts
crypto_df.count()

CoinName           685
Algorithm          685
ProofType          685
TotalCoinsMined    685
TotalCoinSupply    685
dtype: int64

In [149]:
# Keep the rows where coins are mined.
mined_coins = crypto_df['TotalCoinsMined'] >0

crypto_df = crypto_df.loc[mined_coins]

crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [150]:
# Checking the row counts
crypto_df.count()

CoinName           532
Algorithm          532
ProofType          532
TotalCoinsMined    532
TotalCoinSupply    532
dtype: int64

In [151]:
# Create a new DataFrame that holds only the cryptocurrency names, 
# and use the crypto_df DataFrame index as the index for this new DataFrame.

# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = pd.DataFrame(crypto_df["CoinName"])
crypto_names_df.head()

,CoinName
,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [152]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head(2)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000


In [153]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df,columns=["Algorithm","ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [155]:
# Using PCA to reduce dimension to three principal components.

# Initialize PCA model
pca = PCA(n_components=3)
pca

# Get two principal components for the iris data.
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.34364481,  1.09197091, -0.55182732],
       [-0.32699274,  1.09216206, -0.55217755],
       [ 2.30308592,  1.66430638, -0.56756837],
       ...,
       [ 0.33064487, -2.29430111,  0.37348583],
       [-0.16355201, -2.2029402 ,  0.37744213],
       [-0.29795457,  0.78851415, -0.27244921]])

In [156]:
# Create a DataFrame with the three principal components.
crypto_pca_df = pd.DataFrame(
                data=X_pca, columns=["PC 1", "PC 2", "PC 3"], index = X.index)

crypto_pca_df.head(2)

,PC 1,PC 2,PC 3
,,,
42,-0.343645,1.091971,-0.551827
404,-0.326993,1.092162,-0.552178


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [157]:
# Create an elbow curve to find the best value for K.
# create empty list to hold inertia values.
inertia = []

# define k values.
k = list(range(1, 11))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [158]:
# Fit the model - Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvplot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [159]:
# Initialize the K-Means model, using k = 4, as it can be seen on the elbow curve above.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)

predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [160]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, crypto_pca_df], axis=1)
clustered_df.head(2)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3
,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343645,1.091971,-0.551827
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326993,1.092162,-0.552178


In [161]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_names_df
clustered_df.head(2)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName
,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343645,1.091971,-0.551827,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326993,1.092162,-0.552178,404Coin


In [162]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = predictions
clustered_df.head(2)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343645,1.091971,-0.551827,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326993,1.092162,-0.552178,404Coin,0


In [163]:
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(2)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343645,1.091971,-0.551827,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.326993,1.092162,-0.552178,404Coin,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [165]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"])

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [167]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [168]:
# Print the total number of tradable cryptocurrencies.
print(len(clustered_df))

532


In [170]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mms = MinMaxScaler()
clustered_df_scaled = mms.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
clustered_df_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [171]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df=pd.DataFrame(
data=clustered_df_scaled, columns=['TotalCoinSupply','TotalCoinsMined'], index = clustered_df.index)

plot_df

,TotalCoinSupply,TotalCoinsMined
,,
42,4.200000e-11,0.000000e+00
404,5.320000e-04,1.065855e-03
1337,3.141593e-01,2.957551e-02
BTC,2.100000e-05,1.810842e-05
ETH,0.000000e+00,1.087731e-04
...,...,...
ZEPH,2.000000e-03,2.020225e-03
GAP,2.500000e-04,1.508199e-05
BDX,1.400223e-03,9.901351e-04


In [175]:
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df["CoinName"]
plot_df.head(2)


,TotalCoinSupply,TotalCoinsMined,CoinName
,,,
42,4.200000e-11,0.000000,42 Coin
404,5.320000e-04,0.001066,404Coin


In [176]:
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
plot_df["Class"] = clustered_df["Class"]
plot_df.head(2)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
,,,,
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0


In [178]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols=["CoinName"])



:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)